## Support Vector Hyperparameter Tuning (F1-Score)

In [56]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.svm import SVC

import optuna

In [58]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [60]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [62]:
# Hyperparameter tuning

def objective(trial):
    
    params = {
        'C': trial.suggest_float('C', 0.001, 1.0),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    }

    model = SVC(
        **params,
        random_state=42,
        probability=True)
    
    threshold = 0.27
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(f1_score(y_te, y_pred))
        
    return np.mean(scores)

In [64]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-15 19:24:28,305] A new study created in memory with name: no-name-de02149e-a74b-4f98-a664-8d4858a3b428


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-15 19:24:57,333] Trial 0 finished with value: 0.5182642679074513 and parameters: {'C': 0.6740981841562477, 'kernel': 'poly'}. Best is trial 0 with value: 0.5182642679074513.
[I 2024-07-15 19:25:26,200] Trial 1 finished with value: 0.5177061667410501 and parameters: {'C': 0.32086916826147666, 'kernel': 'poly'}. Best is trial 0 with value: 0.5182642679074513.
[I 2024-07-15 19:25:54,112] Trial 2 finished with value: 0.5181882816032892 and parameters: {'C': 0.6271061756410041, 'kernel': 'poly'}. Best is trial 0 with value: 0.5182642679074513.
[I 2024-07-15 19:26:10,848] Trial 3 finished with value: 0.4667738021531367 and parameters: {'C': 0.2573550992640807, 'kernel': 'linear'}. Best is trial 0 with value: 0.5182642679074513.
[I 2024-07-15 19:26:37,145] Trial 4 finished with value: 0.5177061667410501 and parameters: {'C': 0.2966492527160302, 'kernel': 'poly'}. Best is trial 0 with value: 0.5182642679074513.
[I 2024-07-15 19:27:12,211] Trial 5 finished with value: 0.5484785569838

In [69]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=95, state=1, values=[0.55141251958048], datetime_start=datetime.datetime(2024, 7, 15, 20, 43, 39, 448229), datetime_complete=datetime.datetime(2024, 7, 15, 20, 44, 12, 464714), params={'C': 0.020457060411976778, 'kernel': 'rbf'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'C': FloatDistribution(high=1.0, log=False, low=0.001, step=None), 'kernel': CategoricalDistribution(choices=('linear', 'rbf', 'poly'))}, trial_id=95, value=None)
Best hyperparameters: {'C': 0.020457060411976778, 'kernel': 'rbf'}


In [72]:
# See if scaling data improves score

sv = SVC(C=0.020457060411976778, random_state=42, probability=True)

ss = StandardScaler()
mm = MinMaxScaler()

X_train_scaled_ss = pd.DataFrame(ss.fit_transform(X_train))
X_train_scaled_mm = pd.DataFrame(mm.fit_transform(X_train))

scales = [X_train_scaled_ss, X_train_scaled_mm]
names = ['StandardScaler', 'MinMaxScaler']

for i, n in zip(scales, names):
    
    threshold = 0.27
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(i, y_train):
        
        X_tr, X_te = i.iloc[tr],i.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        sv.fit(X_tr, y_tr)
        prob = sv.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(f1_score(y_te, y_pred))
        
    print(f'{n} f1-score: {np.mean(scores)}')

StandardScaler f1-score: 0.5713451671313549
MinMaxScaler f1-score: 0.5572141709954568


The model performs best with the Standard Scaler.